In [2]:
import pandas as pd
import glob
import numpy as np
import pathlib as pl

### 1) Combine all the pks datasets

In [3]:
PATH_TO_PROCESSED_DATA = pl.Path('../datasets/processed')

cases_basic_table_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesBasicTable_excel.xlsx')
cases_development_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesDevelopment_excel.xlsx')
suspects_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]Suspects_excel.xlsx')
len(cases_basic_table_files), len(cases_development_files), len(suspects_files)

(10, 10, 10)

 - first combine all the disticnt tables from all the years to 3 tables throughout the years

#### 1.11 PKS basic table

In [47]:
df = pd.read_excel(cases_basic_table_files[0], skiprows=8, usecols='B:T', header=None,
              names=["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"]) # for sure there is a way how to automate the extraction of the names from various rows, but this seems to be easier

df


,offence or offence category,recorded cases,percentage of all cases,attempts number,attempts %,"< 20,000 inhabitants","20,000 up to 100,000 inhabitants","100,000 up to 500,000 inhabitants","500,000 and more inhabitants",unknown inhabitants,firearm threatened,firearm shot,clearance number,clearance %,suspects count,male suspects,female suspects,non-German suspects number,non-German suspects %
0,total offences,6082064.0,100.0,461117.0,7.6,1404723.0,1602590.0,1261205.0,1755141.0,58405.0,4567.0,5018.0,3336398.0,54.9,2149504.0,1597241.0,552263.0,617392.0,28.7
1,offences against life,2962.0,0.0,1658.0,56.0,912.0,932.0,575.0,529.0,14.0,12.0,112.0,2760.0,93.2,3684.0,3083.0,601.0,947.0,25.7
2,murder (sect. 211 PC),664.0,0.0,415.0,62.5,223.0,182.0,119.0,136.0,4.0,5.0,56.0,633.0,95.3,829.0,699.0,130.0,247.0,29.8
3,other types of murder,602.0,0.0,385.0,64.0,201.0,168.0,109.0,120.0,4.0,3.0,54.0,575.0,95.5,740.0,617.0,123.0,227.0,30.7
4,robbery attended with murder,44.0,0.0,21.0,47.7,14.0,10.0,7.0,13.0,0.0,2.0,1.0,41.0,93.2,75.0,68.0,7.0,19.0,25.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,environmental offences pursuant to supplementa...,14890.0,0.2,137.0,0.9,6508.0,3465.0,1794.0,3049.0,74.0,0.0,437.0,11215.0,75.3,12030.0,9654.0,2376.0,1858.0,15.4
1025,street crime,1342905.0,22.1,77948.0,5.8,266329.0,350467.0,287668.0,434587.0,3854.0,524.0,1350.0,222012.0,16.5,195751.0,173095.0,22656.0,48851.0,25.0
1026,damage to property by graffiti in total,95160.0,1.6,237.0,0.2,20390.0,24380.0,21477.0,27961.0,952.0,0.0,0.0,17871.0,18.8,9793.0,8591.0,1202.0,847.0,8.6
1027,illegal entry/illegal stay under the Aliens Ac...,135743.0,2.2,2293.0,1.7,31095.0,32487.0,23368.0,48328.0,465.0,0.0,0.0,135516.0,99.8,131458.0,97406.0,34052.0,131000.0,99.7


In [52]:
def combine_pks_tables(fnames: list[str], names: list[str], skiprows: int, usecols: str) -> pd.DataFrame:
    table = pd.DataFrame()

    for i, fname in enumerate(fnames):
        print(fname)
        
        year = 2014 + i
        new_names = [f"{year} {n}" for n in names] 
        df_i = pd.read_excel(fname, skiprows=skiprows, usecols=usecols, header=None,
                  names=new_names)
        
        table = pd.concat([table, df_i], axis=1)
    
    return table



In [53]:
basic_table = combine_pks_tables(cases_basic_table_files, ["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"], 8, 'B:T')

basic_table.to_csv(PATH_TO_PROCESSED_DATA / 'pks_basic_table.csv')

basic_table.shape 

../datasets/raw/police_stats\pks2014CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2015CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2016CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2017CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2018CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2019CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2020CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2021CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2022CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2023CasesBasicTable_excel.xlsx


(1139, 190)

#### 1.2 PKS Cases Development

In [50]:
df = pd.read_excel(cases_development_files[0], skiprows=16, usecols='C,D,H', header=None, names=["offence or offence category", "recorded cases", "clearance rate %"])
df

,offence or offence category,recorded cases,clearance rate %
0,total offences,6082064,54.9
1,offences against life,2962,93.2
2,murder (sect. 211 PC),664,95.3
3,other types of murder,602,95.5
4,robbery attended with murder,44,93.2
...,...,...,...
1024,environmental offences pursuant to supplementa...,14890,75.3
1025,street crime,1342905,16.5
1026,damage to property by graffiti in total,95160,18.8
1027,illegal entry/illegal stay under the Aliens Ac...,135743,99.8


In [54]:
# for sure there is a way how to automate the extraction of the names from various rows, but this seems to be easier
cases_development_table = combine_pks_tables(cases_basic_table_files, ["offence or offence category", "recorded cases", "clearance rate %"], 16, 'C,D,H')

cases_development_table.to_csv(PATH_TO_PROCESSED_DATA / 'pks_development_table.csv')

cases_development_table.shape

../datasets/raw/police_stats\pks2014CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2015CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2016CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2017CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2018CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2019CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2020CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2021CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2022CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2023CasesBasicTable_excel.xlsx


(1131, 30)

#### 1.3 Suspects